In [1]:
import time

import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

import IPython.display as ipd

from scipy.io import wavfile as wav

from sklearn import preprocessing
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, precision_score, recall_score, confusion_matrix
import tensorflow as tf
from tensorflow.keras import regularizers
import tensorflow.keras as keras
from tensorflow.keras import Sequential
from tensorflow.keras.layers import *
from tensorflow.keras.regularizers import l2
from tensorflow.keras.callbacks import ReduceLROnPlateau,EarlyStopping,ModelCheckpoint,LearningRateScheduler, Callback

from tqdm import tqdm
import plotly.graph_objects as go



In [2]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())


[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 14442990526158127653
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 16149905408
locality {
  bus_id: 1
  links {
  }
}
incarnation: 12549138427162785627
physical_device_desc: "device: 0, name: Tesla P100-PCIE-16GB, pci bus id: 0000:00:04.0, compute capability: 6.0"
]


2022-11-16 02:40:32.749377: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-11-16 02:40:32.819270: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-16 02:40:32.914153: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-16 02:40:32.914879: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA 

In [3]:
def Alice(plaintext_length, drop = 0., fc_layers = 1, activation = "sigmoid"):
    N = plaintext_length
    plaintext_in = Input((N, ), name = "plaintext_input")
    key_in = Input((N, ), name = "key_input")
    x = concatenate([plaintext_in, key_in])
    for i in range(fc_layers):
        x = Dense(2 * N, activation = activation)(x)
        x = Dropout(drop)(x)
    x = tf.expand_dims(x, -1)
    x = Conv1D(2, 4, strides = 1, padding = "same", activation = activation)(x)
    x = Conv1D(4, 2, strides = 2, padding = "same", activation = activation)(x)
    x = Conv1D(4, 1, strides = 1, padding = "same", activation = activation)(x)
    x = Conv1D(1, 1, strides = 1, padding = "same", activation = "tanh")(x)
    outputs = tf.squeeze(x, axis = -1)
    model = keras.Model(inputs = [plaintext_in, key_in], outputs = outputs)
    return model

def Bob(ciphertext_length, drop = 0., fc_layers = 1, activation = "sigmoid"):
    N = ciphertext_length
    ciphertext_in = Input((N, ), name = "ciphertext_input")
    key_in = Input((N, ), name = "key_input")
    x = concatenate([ciphertext_in, key_in])
    for i in range(fc_layers):
        x = Dense(2 * N, activation = activation)(x)
        x = Dropout(drop)(x)
    x = tf.expand_dims(x, -1)
    x = Conv1D(2, 4, strides = 1, padding = "same", activation = activation)(x)
    x = Conv1D(4, 2, strides = 2, padding = "same", activation = activation)(x)
    x = Conv1D(4, 1, strides = 1, padding = "same", activation = activation)(x)
    x = Conv1D(1, 1, strides = 1, padding = "same", activation = "tanh")(x)
    outputs = tf.squeeze(x, axis = -1)
    model = keras.Model(inputs = [ciphertext_in, key_in], outputs = outputs)
    return model

def Eve(plaintext_length, drop = 0., fc_layers = 1, activation = "sigmoid"):
    N = plaintext_length
    ciphertext_in = Input((N, ), name = "ciphertext_input")
    
    x = ciphertext_in
    for i in range(fc_layers):
        x = Dense(2 * N, activation = activation)(x)
        x = Dropout(drop)(x)
    x = tf.expand_dims(x, axis = -1)
    x = Conv1D(2, 4, strides = 1, padding = "same", activation = activation)(x)
    x = Conv1D(4, 2, strides = 2, padding = "same", activation = activation)(x)
    x = Conv1D(4, 1, strides = 1, padding = "same", activation = activation)(x)
    x = Conv1D(1, 1, strides = 1, padding = "same", activation = "tanh")(x)
    outputs = tf.squeeze(x, axis = -1)
    model = keras.Model(inputs = ciphertext_in, outputs = outputs)
    return model


In [4]:
def get_random_batch(batch_size, string_length):
    return tf.cast(tf.random.uniform((batch_size, string_length), minval = 0, maxval = 2, dtype = "int32") * 2 - 1, "float32")
plaintext = get_random_batch(1, 64)
key =  get_random_batch(1, 64)
print(key)

2022-11-16 02:40:35.292649: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-16 02:40:35.293650: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-16 02:40:35.294757: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-16 02:40:35.297374: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-16 02:40:35.298596: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from S

tf.Tensor(
[[-1. -1. -1. -1.  1.  1. -1.  1. -1.  1. -1. -1.  1. -1. -1. -1. -1. -1.
  -1. -1.  1. -1.  1.  1.  1. -1. -1. -1.  1. -1. -1. -1. -1. -1. -1. -1.
  -1.  1.  1.  1. -1.  1.  1.  1.  1. -1.  1. -1.  1.  1. -1.  1. -1. -1.
   1. -1. -1.  1. -1. -1.  1.  1. -1. -1.]], shape=(1, 64), dtype=float32)


In [5]:
alice = Alice(64)
alice([plaintext, key])

2022-11-16 02:40:37.545329: I tensorflow/stream_executor/cuda/cuda_dnn.cc:369] Loaded cuDNN version 8005


<tf.Tensor: shape=(1, 64), dtype=float32, numpy=
array([[0.5099815 , 0.51004076, 0.5086091 , 0.5064454 , 0.50813246,
        0.51022863, 0.5069851 , 0.5105663 , 0.5076836 , 0.51151586,
        0.5085466 , 0.5070672 , 0.51045567, 0.5062839 , 0.51007587,
        0.50795454, 0.50920135, 0.5097798 , 0.50778204, 0.5099821 ,
        0.50808054, 0.50676185, 0.5110092 , 0.5070825 , 0.5109938 ,
        0.5098831 , 0.5106025 , 0.5066263 , 0.50901234, 0.5082018 ,
        0.51000094, 0.50957006, 0.5097047 , 0.5100066 , 0.50838745,
        0.5082678 , 0.5073172 , 0.5098672 , 0.51086056, 0.5077518 ,
        0.50931484, 0.50949895, 0.5087959 , 0.5070169 , 0.5092565 ,
        0.50803465, 0.50794685, 0.5104311 , 0.5085835 , 0.509046  ,
        0.5103982 , 0.5085596 , 0.50815874, 0.5127613 , 0.506406  ,
        0.50802207, 0.5103933 , 0.5076515 , 0.51109064, 0.5082543 ,
        0.50889355, 0.5109269 , 0.50782096, 0.5051253 ]], dtype=float32)>

In [15]:
class L1(keras.losses.Loss):
    def __init__(self):
        super().__init__()
    def call(self, y, y_pred):
        return tf.reduce_mean(tf.reduce_sum(tf.abs(y - y_pred) / 2, axis = 1), axis = 0)
    def batch_loss(self, y, y_pred):
        return tf.reduce_sum(tf.abs(y - y_pred) / 2, axis = 1)
    
class AliceBobLoss(keras.losses.Loss):
    def __init__(self, N):
        super().__init__()
        self.N = N
    def update_eve_loss(self, eve_loss):
        self.eve_loss = eve_loss
    def call(self, y, y_bob):
        L1_comp = tf.reduce_mean(tf.reduce_sum(tf.abs(y - y_bob) / 2, axis = 1))
        eve_comp = tf.reduce_mean(tf.square(self.N / 2 - self.eve_loss)) / ((self.N /2) **2)
        return L1_comp + eve_comp
        
        
def mean_reconstruction_accuracy(y, y_pred):
    y_pred = tf.cast(y_pred > 0, "float32") * 2 - 1
    return tf.reduce_mean(1 - tf.abs(y - y_pred)/2).numpy()
    
def train_loop(alice, bob, eve,
               optimizer,
                learning_rate = 0.0008,
                num_batches = 10000,
                alice_bob_batches = 1,
                eve_batches = 2,
                batch_size = 4096,
                string_length = 64,
                verbose = False
                ):
    alice_bob_batch_count = 0
    eve_batch_count = 0
    L1_loss = L1()
    alice_bob_loss =  AliceBobLoss(string_length)
    history = {"bob_recon_acc": [],
              "eve_recon_acc": [],
              "eve_loss":[],
              "ab_loss": []}
    for i in tqdm(range(num_batches)):
        with tf.GradientTape(persistent = True) as tape:
            P = get_random_batch(batch_size, string_length)
            K = get_random_batch(batch_size, string_length)

            C = alice([P, K])
            P_bob = bob([C, K])
            P_eve = eve(C)
            
            bob_recon_acc = mean_reconstruction_accuracy(P, P_bob)
            eve_recon_acc = mean_reconstruction_accuracy(P, P_eve)
            eve_loss = L1_loss(P, P_eve)
            alice_bob_loss.update_eve_loss(eve_loss)
            ab_loss = alice_bob_loss(P, P_bob)

            if alice_bob_batch_count < alice_bob_batches:

                alice_grads = tape.gradient(ab_loss, alice.trainable_weights)
                bob_grads = tape.gradient(ab_loss, bob.trainable_weights)
                optimizer.apply_gradients(zip(alice_grads, alice.trainable_weights))
                optimizer.apply_gradients(zip(bob_grads, bob.trainable_weights))
                alice_bob_batch_count += 1
                
            elif eve_batch_count < eve_batches:
                grads = tape.gradient(eve_loss, eve.trainable_weights)
                optimizer.apply_gradients(zip(grads,eve.trainable_weights))
                eve_batch_count +=1
                
            else:
                alice_bob_batch_count = 0
                eve_batch_count = 0
                alice_grads = tape.gradient(ab_loss, alice.trainable_weights)
                bob_grads = tape.gradient(ab_loss, bob.trainable_weights)
                
                optimizer.apply_gradients(zip(alice_grads, alice.trainable_weights))
                optimizer.apply_gradients(zip(bob_grads, bob.trainable_weights))
                alice_bob_batch_count += 1
        del tape
        history["bob_recon_acc"].append(bob_recon_acc)
        history["eve_recon_acc"].append(eve_recon_acc)
        history["eve_loss"].append(eve_loss)
        history["ab_loss"].append(ab_loss)
        if verbose and i % 200 == 0:
            
            print("Batch: ", str(i))
            print("Bob's reconstruction accuracy: {:.2f}%".format(bob_recon_acc * 100))
            print("Eve's reconstruction accuracy: {:.2f}%".format(eve_recon_acc * 100))
            print("Bob and Alice's loss: {:.2f}".format(ab_loss.numpy()))
            print("Eve's loss: {:.2f}".format(eve_loss.numpy()))
    
    return history, alice, bob, eve
    

In [16]:
def run_experiment(fc_layers = 1,
                drop = 0.,
                activation = "sigmoid",
                learning_rate = 0.0008,
                num_batches = 10000,
                alice_bob_batches = 1,
                eve_batches = 2,
                batch_size = 4096,
                string_length = 64,
                  ):
    
    optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
    
    alice = Alice(string_length,fc_layers = fc_layers, drop= drop, activation = activation)
    bob = Bob(string_length,fc_layers = fc_layers, drop= drop, activation = activation)
    eve = Eve(string_length,fc_layers = fc_layers, drop= drop, activation = activation)
    
    return train_loop(alice, bob, eve,
               optimizer,
                learning_rate,
                num_batches,
                alice_bob_batches,
                eve_batches,
                batch_size,
                string_length,
                )

In [17]:
base_alice, base_bob, base_eve, base_models_history = run_experiment(num_batches = 25000, string_length = 16)

  0%|          | 5/25000 [00:00<16:33, 25.15it/s]

Batch:  0
Bob's reconstruction accuracy: 49.84%
Eve's reconstruction accuracy: 49.84%
Bob and Alice's loss: 8.01
Eve's loss: 8.01


  1%|          | 206/25000 [00:07<14:07, 29.24it/s]

Batch:  200
Bob's reconstruction accuracy: 50.17%
Eve's reconstruction accuracy: 50.17%
Bob and Alice's loss: 7.99
Eve's loss: 7.99


  2%|▏         | 406/25000 [00:14<14:22, 28.52it/s]

...

 98%|█████████▊| 24604/25000 [14:44<00:14, 28.09it/s]

Batch:  24600
Bob's reconstruction accuracy: 98.45%
Eve's reconstruction accuracy: 53.81%
Bob and Alice's loss: 0.25
Eve's loss: 7.39


 99%|█████████▉| 24805/25000 [14:51<00:06, 29.49it/s]

Batch:  24800
Bob's reconstruction accuracy: 98.48%
Eve's reconstruction accuracy: 53.80%
Bob and Alice's loss: 0.25
Eve's loss: 7.39


100%|██████████| 25000/25000 [14:58<00:00, 27.84it/s]
